In [ ]:
import src.tools.check_bank_balance
print(dir(src.tools.check_bank_balance.run))

In [1]:
import json

In [2]:
def extract_sql_schema(table_meta):

    table = table_meta["name"]
    schema = table_meta.get("schema")

    cols = []

    for f in table_meta.get("fields", []):
        col = f"{f['name']} {f['database_type'].upper()}"

        if f.get("semantic_type") == "type/PK":
            col += " PRIMARY KEY"

        cols.append(col)

    # ✅ only add schema if it exists
    table_name = f"{schema}.{table}" if schema else table

    return f"""CREATE TABLE {table_name} (
  {", ".join(cols)}
);"""

In [ ]:
import requests

url = "https://mwt-mb.transxt.in/api/database/7/metadata"

payload = ""
headers = {
  'X-Metabase-Session': '',
  'Cookie': ''
}

response = requests.request("GET", url, headers=headers, data=payload)



{"description":null,"features":["basic-aggregations","standard-deviation-aggregations","expression-aggregations","foreign-keys","right-join","left-join","native-parameters","nested-queries","expressions","set-timezone","binning","inner-join","advanced-math-expressions"],"cache_field_values_schedule":"0 0 23 * * ? *","timezone":"Asia/Calcutta","auto_run_queries":false,"metadata_sync_schedule":"0 2 * * * ? *","name":"Suryoday-UPIMerchant","caveats":null,"tables":[{"description":null,"entity_type":"entity/GenericTable","schema":null,"show_in_getting_started":false,"name":"accno","fields":[{"description":null,"database_type":"VARCHAR","semantic_type":null,"table_id":1159,"coercion_strategy":null,"name":"accno","fingerprint_version":5,"has_field_values":"search","settings":null,"caveats":null,"fk_target_field_id":null,"updated_at":"2025-10-21T12:02:06.804","custom_position":0,"effective_type":"type/Text","active":true,"parent_id":null,"id":21930,"last_analyzed":"2025-10-21T12:02:06.896","po

In [12]:

SCHEMA = ""
for table in json.loads(response.text)["tables"]:
    SCHEMA += extract_sql_schema(table) + f"\n"


In [14]:
print(SCHEMA)

CREATE TABLE accno (
  accno VARCHAR
);
CREATE TABLE aggregator_details (
  aggregator_id INT PRIMARY KEY, aggunique_id VARCHAR, aggregator_name VARCHAR, encryption_key VARCHAR, callbackurl_1 VARCHAR, callbackurl_2 VARCHAR, created_at TIMESTAMP, updated_at TIMESTAMP, is_active INT
);
CREATE TABLE apiaccess (
  apiaccessid INT UNSIGNED PRIMARY KEY, mid INT, apiaccess VARCHAR, accesstype INT, updatedon DATETIME, insertedon DATETIME, isactive INT, nodeip VARCHAR
);
CREATE TABLE apiauditlog (
  apiauditlogid INT PRIMARY KEY, reqid VARCHAR, userid INT, serviceid INT, reqintime DATETIME, reqouttime DATETIME, channel INT, createddate TIMESTAMP, sessionid VARCHAR, node VARCHAR, udf1 VARCHAR, udf2 VARCHAR, udf3 VARCHAR, udf4 VARCHAR, udf5 VARCHAR, udf6 VARCHAR, udf7 VARCHAR, apiurl VARCHAR, payload MEDIUMTEXT, errorcode VARCHAR
);
CREATE TABLE apimaster (
  apimasterid INT UNSIGNED PRIMARY KEY, apikey VARCHAR, apiname VARCHAR, apidesc VARCHAR, isactive INT, insertedon DATETIME, nodeip VARCHAR
)

In [ ]:
SYSTEM_PROMPT = f""" 
You are an AI assistant that converts natural language questions into SQL queries. A database schema will be provided to you. Follow these rules strictly:
- Always return a single SQL query as your final output. Do not include explanations, descriptions, comments, or any additional text. Only return SQL.
- Use only the tables and columns explicitly provided in the schema. If the user asks for information that does not exist in the schema, return a SQL query that uses only the available fields in the most reasonable way.
- Do not invent new tables, columns, or relationships. Use only what is defined.
- When joins are required, infer the relationship based on foreign-key naming conventions or explicit schema instructions. If ambiguity exists, choose the relationship that is most consistent with typical relational database design.
- Fully qualify columns when necessary to avoid ambiguity.
- Never return placeholders. For example, do not return "table_name" or "column_name". Always return actual schema elements.
- If multiple SQL interpretations are possible, choose the simplest valid SQL that answers the user's question.
- Use standard ANSI SQL unless the schema or user explicitly requests a specific dialect.
- Do not include LIMIT clauses unless explicitly requested.
- Ensure your SQL is syntactically correct and ready to execute.
- DONT USE UNNECESSARILY COMPLEX COMMANDS. RESPOND WITH THE MOST UNDERSTANABLE AND USER READABLE COMMANDS.
Use the schema provided below for your query output:
{SCHEMA}
"""
QUERY = "Retrieve all virtual account transaction records between Feb 1 and Feb 19, 2026 ordered by corporate ID and latest transaction date."


In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(api_key=os.getenv("OPEN_AI_API_KEY"))
response = client.responses.create(
    model="gpt-5",
    input=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": QUERY}
    ]
)

In [ ]:
def maintain_context_limit(
    data,
    max_total_chars: int = 2500,
    max_field_value: int = 500,
    max_rows: int = 200,
) -> str:
    ''' 
    1. Takes in data (string or API response) and converts to JSON.
    2. Caps total rows/fields at max_rows.
    3. Truncates any field value above max_field_value (replaces with "output too large to display").
    4. Truncates overall output if total chars > max_total_chars.
    5. Returns reduced JSON as a string.
    '''

    # Step 1 — convert input to JSON-safe data
    try:
        if isinstance(data, str):
            json_data = json.loads(data)
        else:
            json_data = data
    except Exception:
        return "An error occured in processing the API response"

    # Ensure we are always working with a list for max_rows logic
    if isinstance(json_data, dict):
        items = [json_data]
    elif isinstance(json_data, list):
        items = json_data
    else:
        items = [{"response": str(json_data)}]

    # Step 2 — Cap number of rows
    if len(items) > max_rows:
        items = items[:max_rows]

    # Step 3 — Truncate long field values
    def truncate_values(obj):
        if isinstance(obj, dict):
            return {
                k: truncate_values(v)
                for k, v in obj.items()
            }
        elif isinstance(obj, list):
            return [truncate_values(v) for v in obj]
        elif isinstance(obj, str):
            if len(obj) > max_field_value:
                return "output too large to display"
            return obj
        else:
            return obj

    items = truncate_values(items)

    # Step 4 — Convert to JSON string and check total char limit
    output = json.dumps(items, indent=2)

    if len(output) > max_total_chars:
        truncated = output[:max_total_chars] + "\n... output truncated ..."
        return truncated

    return output


In [ ]:
import json
maintain_context_limit(json.loads("""{
  "data": {
    "branchId": "",
    "branchDescription": "",
    "dtlStatementAddressList": [],
    "dtlStatementTxnInfo": []
  }
}"""))

In [ ]:
from Crypto.Cipher import AES
import base64
import os
from dotenv import load_dotenv
load_dotenv()

def decrypt(encrypted_data: str, key: str = os.getenv("decryption_key", "")) -> str:
    if encrypted_data is None:
        return None

    # AES requires keys of length 16, 24, or 32 bytes
    key_bytes = key.encode('utf-8') 

    cipher = AES.new(key_bytes, AES.MODE_ECB)
    decrypted_bytes = cipher.decrypt(base64.b64decode(encrypted_data))

    # Remove PKCS7 padding
    padding_length = decrypted_bytes[-1]
    decrypted_bytes = decrypted_bytes[:-padding_length]

    return decrypted_bytes.decode('utf-8')

In [ ]:
decrypt("DYi7gkCWJ9T3XN4r0SFoXe1QhPAvKzyslLayAX5J9XU=")

In [ ]:
import pandas as pd

dataframe = pd.read_csv("src/api_pricing.csv")

USD_TO_INR = 83.0
PRICE_UNIT = 1_000_000   # OpenAI prices are per 1M tokens
model = "gpt-4o"


def clean_price(value):
    if isinstance(value, str):
        value = value.replace("$", "").strip()
    return float(value)


def get_cost(input: str, output: str,
             dataframe=dataframe,
             model: str = model) -> str:

    required_cols = {"Model", "Input", "Output"}
    missing = required_cols - set(dataframe.columns)
    if missing:
        raise ValueError(f"Missing columns: {missing}")

    rows = dataframe.loc[
        dataframe["Model"] == model,
        ["Input", "Output"]
    ]

    if rows.empty:
        return "₹0.00"

    # ✅ USD price per 1M tokens
    input_price_usd = clean_price(rows["Input"].iloc[0])
    output_price_usd = clean_price(rows["Output"].iloc[0])

    # ✅ Convert to INR per token
    input_price_per_token = (input_price_usd * USD_TO_INR) / PRICE_UNIT
    output_price_per_token = (output_price_usd * USD_TO_INR) / PRICE_UNIT

    # Approx tokens
    input_tokens = len(input.split())
    output_tokens = len(output.split())

    cost = (
        input_tokens * input_price_per_token +
        output_tokens * output_price_per_token
    )

    return f"₹{cost:.6f}"

In [ ]:
get_cost(
    "Hello how are you",
    "I am doing great"
)

In [ ]:
import pandas as pd

dataframe = pd.read_csv("src/api_pricing.csv")

# Clean currency columns
for col in ["Input", "Output"]:
    dataframe[col] = (
        dataframe[col]
        .astype(str)
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
        .str.strip()
        .astype(float)
    )

print(dataframe.dtypes)

In [ ]:
dataframe.to_csv("src/api_pricing.csv", index=False)